# Agentic RAG Playground by LlamaIndex

In [1]:
import datasets
from llama_index.core.schema import Document

In [2]:
# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        text="\n".join([
            f"Name: {guest_dataset['name'][i]}",
            f"Relation: {guest_dataset['relation'][i]}",
            f"Description: {guest_dataset['description'][i]}",
            f"Email: {guest_dataset['email'][i]}"
        ]),
        metadata={"name": guest_dataset['name'][i]}
    )
    for i in range(len(guest_dataset))
]

In [3]:
# Initialize the tool
from src.tools import GuestInfoRetrieverTool
guest_info_tool = GuestInfoRetrieverTool(docs)

In [ ]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.ollama import Ollama
from llama_index.core.tools import FunctionTool
from src.tools import GuestInfoRetrieverTool
from src.utils import ensure_ollama_server

# Initialize the guest info retriever tool
guest_info_retriever = GuestInfoRetrieverTool(docs)

# Create a proper FunctionTool
guest_info_tool = FunctionTool.from_defaults(
    fn=guest_info_retriever.get_guest_info,
    name="guest_info",
    description="Retrieve information about guests attending the gala by their name"
)

# Ensure Ollama server is running
await ensure_ollama_server()

# Initialize the LLM
llm = Ollama(model="llama3:latest")

# Create the agent with minimal configuration
alfred = ReActAgent.from_tools(
    [guest_info_tool],
    llm=llm,
    verbose=True,    # True: visible to thought & action steps
    max_iterations=0  # to limit how many iterations to get the final result.
)

# Example query
try:
    response = await alfred.aquery("Tell me about our guest named 'Lady Ada Lovelace'.")
    # Check if the response contains the answer
    if "Lady Ada Lovelace" in response.response:
        print("🎩 Alfred's Response:")
        print(response.response)
    else:
        print("🎩 Alfred did not find the answer.")
except Exception as e:
    print(f"An error occurred: {e}")


Ollama server is running and ready.
Ollama server is already running.
> Running step 26ce435e-624a-490f-b69f-05afe98ca90e. Step input: Tell me about our guest named 'Lady Ada Lovelace'.
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: guest_info
Action Input: {'query': 'Lady Ada Lovelace'}
Observation: Error: name 're' is not defined
> Running step e13cd42e-04e5-4fc0-a1b6-5c46b3b3edf2. Step input: None


In [ ]:
🎩 Alfred's Response:
Lady Ada Lovelace is an esteemed mathematician and friend, renowned for her pioneering work in mathematics and computing. She is celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine. Her email is ada.lovelace@example.com.

🎩 Alfred's Response:
Lady Ada Lovelace is indeed a renowned mathematician and friend, celebrated for her pioneering work in mathematics and computing. She is often referred to as the first computer programmer due to her work on Charles Babbage's Analytical Engine.